# Preparing our data for use in Mobie 

In [1]:
import os 
from glob import glob
import tifffile
import zarr 
from tqdm import tqdm

In [6]:
# Setting directories
root_dir = '/mnt/efs/shared_data/instance_no_gt/20230830_TIF_cellpose_test/'
mask_files = sorted(glob(os.path.join(root_dir, 'masks', '*.tif')))
raw_tiles = sorted(glob(os.path.join(root_dir, 'raw_files', '*.tif')))

### Stitch

In [7]:
image_test = tifffile.imread(raw_tiles[599])
tile_shape = image_test.shape

# getting the full shape 
tiles_y = 26
tiles_x = 37
tiles_x * tiles_y == len(raw_tiles) # True !

# computing big shape
big_shape = (tiles_x * image_test.shape[0], tiles_y * image_test.shape[1])
print(big_shape) # 37440, 71040 pixel image; why we need .zarr


(53280, 49920)


In [2]:
# define PATH
path_to_stitch = '/mnt/efs/shared_data/instance_no_gt/big_zarr/stitched_image_plane_6.zarr'

# create new .zarr 
zarrfile = zarr.open(path_to_stitch, mode = 'a')
zarr_image = zarrfile.create_dataset('image', shape = big_shape, chunks = image_test.shape, dtype = image_test.dtype)

NameError: name 'big_shape' is not defined

In [11]:
zarr_image = zarrfile["image"]

In [13]:
# Defining sub-structures of the .zarr file 

# First, need to arrange the

for idx, image in tqdm(enumerate(raw_tiles), total = len(raw_tiles), disable=False): 
    tile_index_y = idx // tiles_y
    tile_index_x = idx % tiles_x 

    image = tifffile.imread(image)
    
    # translate tile indices to actual coordinate
    index_y = tile_index_y * tile_shape[0]
    index_x = tile_index_x * tile_shape[1]
    # print(idx, ":", tile_index_y, tile_index_x, ":", index_y, index_x)
    # continue
    
    # time to write to .zarr
    zarr_image[index_y:(image.shape[0]+index_y),index_x:(image.shape[1]+index_x)] = image 


 70%|██████████████████████████████████████████████████████████████████                            | 676/962 [01:12<00:30,  9.32it/s]


ValueError: parameter 'value': expected array with shape (0, 1920), got (1440, 1920)

In [ ]:
# looking at it via napari
import napari 
image = zarrfile['image']
napari.view_image(image)
napari.run()

ImportError: No Qt bindings could be found.

napari requires either PyQt5 or PySide2 to be installed in the environment.
To install the default backend (currently PyQt5), run "pip install napari[all]" 
You may also use "pip install napari[pyside2]"for Pyside2, or "pip install napari[pyqt5]" for PyQt5